In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# imports
from copy import deepcopy
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer

from typing import Iterable

In [3]:
# parse the dataset with CountVectorizer
# since CountVectorizer works with a list of strings, we need to turn df into a list of strings
# # there is a problem the symptom may consist of two words 
# let's turn them into one
# function takes a string (the name of the symptom) and replaces and translates it into snake style

def string_preparation(st:str) -> str:
    return  st.replace('(',' ')\
            .replace(')',' ')\
            .strip()\
            .replace(' ','_')\
            .replace('__','_')
# for processing strings
def row_to_string(row: Iterable['str'] ) -> str:
    return ' '.join(row.values)

df = pd.read_csv('/content/dataset.csv',delimiter=",")

df_new = (df.drop('Disease', axis = 1).fillna('')).applymap(string_preparation)

simptoms_list = df_new.apply(row_to_string, axis=1).values.tolist()

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(simptoms_list)

my_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names_out())
symptoms_list = vectorizer.get_feature_names_out()

my_df["Disease"] = df["Disease"].map(str.strip)
# we have received data that we can feed to the ML algorithm
my_df.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset.csv'

In [ ]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.3 MB/s eta 0:00:00


In [ ]:
import re
from spellchecker import SpellChecker
 
spell = SpellChecker()

for dis in my_df["Disease"].unique():
    for word in re.split(pattern = r'[\s()]+', string = dis, maxsplit=0, flags=0):
        if word not in ["", "GERD", 'cholestasis', 'spondylosis', 'Gastroenteritis',
                        'Hyperthyroidism', 'Hypothyroidism']\
        and len(word) > 1\
        and spell.correction(word)!=word:
            print ('word ', word)  
            print(spell.correction(word))  
            print ("dis*   ", dis) 

word  diseae
disease
dis*    Peptic ulcer diseae
word  Dimorphic
morphic
dis*    Dimorphic hemmorhoids(piles)
word  hemmorhoids
hemorrhoids
dis*    Dimorphic hemmorhoids(piles)
word  Osteoarthristis
None
dis*    Osteoarthristis
word  Paroymsal
None
dis*    (vertigo) Paroymsal  Positional Vertigo


In [ ]:
# compiling a dictionary of error replacement
errors_dict = {
    'Peptic ulcer diseae': 'Peptic ulcer disease',
    'Dimorphic hemmorhoids(piles)': 'Dimorphic hemorrhoids (piles)',
    'Osteoarthristis': 'Osteoarthritis',
    '(vertigo) Paroymsal  Positional Vertigo': '(vertigo) Paroxysmal  Positional Vertigo'
}
# обрабатываем
def disease_prep(disease):
    if disease in errors_dict:
         return errors_dict[disease]
    return disease
my_df['Disease'] =my_df['Disease'].map(disease_prep)

In [ ]:
# проверяем, checking
for dis in my_df["Disease"].unique():
    for word in re.split(pattern = r'[\s()]+', string = dis, maxsplit=0, flags=0):
        if word not in ["", "GERD", 'cholestasis', 'spondylosis', 'Gastroenteritis', 'Osteoarthritis',
                        'Hyperthyroidism', 'Hypothyroidism', 'Dimorphic', 'Paroxysmal']\
        and len(word) > 1\
        and spell.correction(word)!=word:
            print ('word ', word)  
            print(spell.correction(word))  
            print ("dis*   ", dis)

In [ ]:
for sym in my_df.columns:
    for word in sym.split('_'):
        if word not in ["", "diarrhoea", 'polyuria', 'sensorium', 'breathlessness',
                       'micturition', 'mucoid', 'unsterile', 'unsteadiness']\
        and spell.correction(word)!=word:
            print (word)
            print(spell.correction(word))
            print ("sym*   ", sym)

dischromic
None
sym*    dischromic_patches
scurring
scarring
sym*    scurring
extremeties
extremities
sym*    swollen_extremeties
typhos
typhus
sym*    toxic_look_typhos


In [ ]:
# It 's not clear what scurrying is
# let's take a closer look
my_df[my_df['scurring']==1]['Disease'].unique()
#  scarring 100500

array(['Acne'], dtype=object)

In [ ]:
# It 's not clear what toxic_look_typhos is
# let's take a closer look
my_df[my_df['toxic_look_typhos']==1]['Disease'].unique()
# toxic_look_typhus

array(['Typhoid'], dtype=object)

In [ ]:
#compiling a dictionary of error replacement
symptoms_errors_dict = {
    'cold_hands_and_feets': 'cold_hands_and_feet',
    'dischromic_patches': 'dyschromic_patches',
    'scurring': 'scarring',
    'swollen_extremeties': 'swollen_extremities',
    'toxic_look_typhos':'toxic_look_typhus'
}
def symptom_prep(symptom):
    if symptom in symptoms_errors_dict:
         return symptoms_errors_dict[symptom]
    return symptom
my_df.columns=my_df.columns.map(symptom_prep)

In [ ]:
for sym in my_df.columns:
    for word in sym.split('_'):
        if word not in ["", "diarrhoea", 'polyuria', 'sensorium', 'breathlessness',
                       'micturition', 'mucoid', 'unsterile', 'unsteadiness', 'dyschromic']\
        and spell.correction(word)!=word:
            print (word)
            print(spell.correction(word))
            print ("sym*   ", sym)

In [ ]:
my_df.groupby('Disease').Disease.count()

Disease
(vertigo) Paroxysmal  Positional Vertigo    120
AIDS                                        120
Acne                                        120
Alcoholic hepatitis                         120
Allergy                                     120
Arthritis                                   120
Bronchial Asthma                            120
Cervical spondylosis                        120
Chicken pox                                 120
Chronic cholestasis                         120
Common Cold                                 120
Dengue                                      120
Diabetes                                    120
Dimorphic hemorrhoids (piles)               120
Drug Reaction                               120
Fungal infection                            120
GERD                                        120
Gastroenteritis                             120
Heart attack                                120
Hepatitis B                                 120
Hepatitis C                     

In [ ]:
# Let's look for duplicates
my_df[my_df.duplicated ()].shape

(4616, 132)

In [ ]:
df_drop_dup = my_df.drop_duplicates() 
print(f' without duplicated shape  {df_drop_dup.shape}')
df_drop_dup.groupby('Disease').Disease.count().sort_values()

 without duplicated shape  (304, 132)


Disease
Gastroenteritis                              5
AIDS                                         5
Acne                                         5
Urinary tract infection                      5
Allergy                                      5
Fungal infection                             5
Heart attack                                 5
Paralysis (brain hemorrhage)                 5
Arthritis                                    6
Cervical spondylosis                         6
Drug Reaction                                6
Impetigo                                     6
Hypertension                                 6
Dimorphic hemorrhoids (piles)                6
(vertigo) Paroxysmal  Positional Vertigo     7
Psoriasis                                    7
Osteoarthritis                               7
Peptic ulcer disease                         7
Hepatitis C                                  7
Bronchial Asthma                             7
GERD                                         7
Malar

In [ ]:
# import models
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

X = df_drop_dup.drop("Disease", axis=1)
y = df_drop_dup["Disease"]


n_fold = 7

# let's define the parameters of cross-validation (stratified 7-stock with mixing)
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state= 11)

In [ ]:
clf  = DecisionTreeClassifier(random_state=11)

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [0.56818182 0.45454545 0.54545455 0.55813953 0.69767442 0.53488372
 0.65116279]
mean scores  0.57


In [ ]:
clf  = LogisticRegression(random_state=11)

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [1. 1. 1. 1. 1. 1. 1.]
mean scores  1.0


In [ ]:
clf  = KNeighborsClassifier()
 
scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', *scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  1.0 1.0 1.0 1.0 1.0 1.0 1.0
mean scores  1.0


In [ ]:
clf  = GaussianNB()

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [1.         1.         0.97727273 0.97674419 1.         1.
 1.        ]
mean scores  0.99


In [ ]:
clf  = SVC(random_state=11)
 
scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [1. 1. 1. 1. 1. 1. 1.]
mean scores  1.0


In [ ]:
clf  = RandomForestClassifier(random_state=11)

scores: np.array = cross_val_score(clf, X, y, cv=skf)
print ('all scores ', scores)
print ('mean scores ', round(scores.mean(), 2))

all scores  [1. 1. 1. 1. 1. 1. 1.]
mean scores  1.0


In [ ]:
# ask user for symptoms input
user_symptoms = input("Enter your symptoms separated by commas: ")

# clean the user input
user_symptoms = user_symptoms.strip().lower().replace(" ", "_")

# check that user input is valid
for symptom in user_symptoms.split(","):
    if symptom not in my_df.columns:
        print(f"Invalid symptom entered: {symptom}")
        break
else:
    # create a numpy array from the user input symptoms
    user_symptoms_array = np.zeros(len(my_df.columns) - 1)
    for symptom in user_symptoms.split(","):
        user_symptoms_array[my_df.columns.get_loc(symptom)] = 1

    # calculate the cosine similarity between the user input and the dataset
    similarity_scores = my_df.iloc[:, :-1].apply(lambda row: np.dot(row, user_symptoms_array) / (np.linalg.norm(row) * np.linalg.norm(user_symptoms_array)), axis=1)

    # get the top 5 diseases that match the user's symptoms
    top_diseases = similarity_scores.nlargest(5)

    # output the top 5 diseases
    print("Top 5 matching diseases:")
    
    for disease in top_diseases.index:
        print(disease)


Enter your symptoms separated by commas: patches_in_throat,high_fever,extra_marital_contacts
Top 5 matching diseases:
61
67
471
477
881


In [ ]:
# input the symptoms
user_symptoms = input("Please enter your symptoms separated by commas: ")
# prepare the user input
user_symptoms = [string_preparation(symptom.strip()) for symptom in user_symptoms.split(",")]

# check if the input symptoms are in the dataframe
missing_symptoms = set(user_symptoms) - set(my_df.columns)
if missing_symptoms:
    print(f"Sorry, we don't have any records for the following symptoms: {missing_symptoms}")
else:
    # filter the dataframe
    filtered_df = my_df.loc[:, ["Disease"] + user_symptoms]
    filtered_df = filtered_df[(filtered_df[user_symptoms].sum(axis=1) == len(user_symptoms))]
    # get the most common disease
    if filtered_df.empty:
        print("Sorry, we could not find any diseases that match your symptoms.")
    else:
        disease = filtered_df["Disease"].value_counts().idxmax()
        print(f"The most likely disease is {disease}.")

Please enter your symptoms separated by commas: patches_in_throat,high_fever,extra marital contacts
The most likely disease is AIDS.


In [ ]:
!pip install PySimpleGUI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 11.1 MB/s eta 0:00:00


In [ ]:
import PySimpleGUI as sg

def next_screen(NAME,user_symptoms):
        
    sg.theme('DarkBlue7')
    
    layout1 = [
        [sg.Text('                                      '),sg.Text('Disease Prediction!', font=('Futura', 20))],
        [sg.Text('')],
        [sg.Text('')],
        [sg.Text('                  '),sg.Text(f'Name: {NAME}', font=('Futura', 14))],
        [sg.Text('')],
        [sg.Text('        '),sg.Text('Disease Predicted: ', font=('Futura', 14)), sg.Input(key='-pred-', size=(30, 1),background_color='lightblue',text_color='Black')],
        [sg.Text('')],
        [sg.Text('')],
        [sg.Text('                                        '),sg.Button('Submit', size=(15, 1))]
    ]
    
    window_1 = sg.Window('Hey Doctor', layout1,size=(500, 300))

    
    # check if the input symptoms are in the dataframe
    missing_symptoms = set(user_symptoms) - set(my_df.columns)
    # if missing_symptoms:
    #     print(f"Sorry, we don't have any records for the following symptoms: {missing_symptoms}")
    # else:
    #     # filter the dataframe
    #     filtered_df = my_df.loc[:, ["Disease"] + user_symptoms]
    #     filtered_df = filtered_df[(filtered_df[user_symptoms].sum(axis=1) == len(user_symptoms))]
    #     # get the most common disease
    #     if filtered_df.empty:
    #         print("Sorry, we could not find any diseases that match your symptoms.")
    #     else:
    #         disease = filtered_df["Disease"].value_counts().idxmax()
    #         print(f"The most likely disease is {disease}.")
    
    while True:
        event, values = window_1.read()
        if event == sg.WINDOW_CLOSED:
            break
        if event == 'Submit':
            if missing_symptoms:

                window_1['-pred-'].update(value=f"Sorry, we don't have any records for the following symptoms: {missing_symptoms}")
            else:
                # filter the dataframe
                filtered_df = my_df.loc[:, ["Disease"] + user_symptoms]
                filtered_df = filtered_df[(filtered_df[user_symptoms].sum(axis=1) == len(user_symptoms))]
                # get the most common disease
                if filtered_df.empty:
                    window_1['-pred-'].update(value="Sorry, we could not find any diseases that match your symptoms.")
                else:
                    disease = filtered_df["Disease"].value_counts().idxmax()
                    window_1['-pred-'].update(value=f"The most likely disease is {disease}.")
            
            
        

    window_1.close()

In [ ]:
import PySimpleGUI as sg

sg.theme('DarkBlue7')

layout = [
    [sg.Text('                                      '),sg.Text('Hey Doctor!', font=('Futura', 20))],
    [sg.Text('')],
    [sg.Text('')],
    [sg.Text('                  '),sg.Text('Name:', font=('Futura', 14)), sg.Input(key='-NAME-', size=(30, 1),background_color='lightblue',text_color='Black')],
    [sg.Text('')],
    [sg.Text('        '),sg.Text('Symptoms:', font=('Futura', 14)), sg.Input(key='-SYMP-', size=(30, 1),background_color='lightblue',text_color='Black')],
    [sg.Text('')],
    [sg.Text('')],
    [sg.Text('                                        '),sg.Button('Submit', size=(15, 1))]
]

window = sg.Window('Hey Doctor', layout,size=(500, 300))

while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    if event == 'Submit':
        name = values['-NAME-']
        symptoms = values['-SYMP-']
        user_symptoms = symptoms
        user_symptoms = [string_preparation(symptom.strip()) for symptom in user_symptoms.split(",")]
        next_screen(name,user_symptoms)
        
        window.close()

window.close()

TclError: ignored